# Usage of Copernicus Dataspace
Compared to the feature rich sentinelhub api, the Copernicus Dataspace is requires only an account and no subscription. Therefore, create an account on [https://dataspace.copernicus.eu/](https://dataspace.copernicus.eu/). For registration, please follow this [guide](https://documentation.dataspace.copernicus.eu/Registration.html).

[Overview of the authentification process](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html)

The API of the Copernicus dataspace is documented on [https://documentation.dataspace.copernicus.eu/APIs.html](https://documentation.dataspace.copernicus.eu/APIs.html).

You navigate to your desired location with the [Copernicus Browser](https://browser.dataspace.copernicus.eu), and copy all the GPS locations and bands you need. 

In [17]:
import time

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session

COPERNICUS_TOKEN_URL = 'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token'

In [18]:
CLIENT_ID = ""
CLIENT_SECRET = ""


def get_api_access_token(client_id, client_secret):
    client = BackendApplicationClient(client_id=client_id)
    oauth = OAuth2Session(client=client)
    token = oauth.fetch_token(token_url=COPERNICUS_TOKEN_URL, client_secret=client_secret, include_client_id=True)
    return token, oauth


token, oauth = get_api_access_token(CLIENT_ID, CLIENT_SECRET)

In [29]:
def token_check(token):
    if token['expires_at'] - time.time() <= 0.0:
        return True
    else:
        return False

## Download a single image as PNG

In [20]:
from .src.evalscripts import enhanced_true_color_evalscript as evalscript

maschsee_coords_wgs84 = (9.733200, 52.342366, 9.755344, 52.363231)

request = {
    "input": {
        "bounds": {
            "properties": { "crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84" },
            "bbox": maschsee_coords_wgs84,
        },
        "data": [
            {
                "type": "sentinel-2-l1c",
                "dataFilter": {
                    "timeRange": {
                        "from": "2022-10-01T00:00:00Z",
                        "to": "2022-10-31T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 512,
        "height": 512,
    },
    "evalscript": evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)

ImportError: attempted relative import with no known parent package

In [21]:
from PIL import Image

with open('./test.png', 'wb') as file:
    file.write(response.content)

In [22]:
true_color_evalscript = """
    //VERSION=3

    function setup() {
        return {
            input: [{
                bands: ["B02", "B03", "B04"]
            }],
            output: {
                bands: 3
            }
        };
    }

    function evaluatePixel(sample) {
        return [sample.B04, sample.B03, sample.B02];
    }
"""

In [23]:
maschsee_coords_wgs84 = (9.733200, 52.342366, 9.755344, 52.363231)
resolution = 10
request = {
    "input": {
        "bounds": {
            "properties": { "crs": "http://www.opengis.net/def/crs/OGC/1.3/CRS84" },
            "bbox": maschsee_coords_wgs84,
        },
        "data": [
            {
                "type": "sentinel-2-l1c",
                "dataFilter": {
                    "timeRange": {
                        "from": "2022-04-01T00:00:00Z",
                        "to": "2022-05-01T00:00:00Z",
                    }
                },
            }
        ],
    },
    "output": {
        "width": 148,
        "height": 234,
    },
    "evalscript": true_color_evalscript,
}

url = "https://sh.dataspace.copernicus.eu/api/v1/process"
response = oauth.post(url, json=request)
if response.status_code == 200:
    with open('./test.png', 'wb') as file:
        file.write(response.content)

    img = Image.open('./test.png')
    img.show()
else:
    print(response)
    print(response.text)

## Usage of WMS services
For the WMS services you need to define a "Configuration Utility" in the [Copernicus Dashboard](https://shapps.dataspace.copernicus.eu/dashboard). This defines what layers can be accessed through your programmed instance. Instructions on doing so can be found [here](https://www.sentinel-hub.com/develop/dashboard/) (although this is sentinel hub, the workflow is essentially the same).
Afterward, you can copy the **instance_id** of for your utility.
The WMS services for the Copernicus dataspace are documented [here](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/OGC/WMS.html).


In [24]:
INSTANCE_ID = "d337d9fe-cb64-4c19-beb2-15b919c5f417"
WMS_URL = f"https://sh.dataspace.copernicus.eu/ogc/wms/{INSTANCE_ID}"

#Example request for getting capabilities of the WMS Services
from bs4 import BeautifulSoup

capabilities = oauth.request(url=f"https://sh.dataspace.copernicus.eu/ogc/wms/{INSTANCE_ID}?REQUEST=GetCapabilities",
                             method="GET")
soup = BeautifulSoup(capabilities.content, 'xml')
print(soup.prettify())

<?xml version="1.0" encoding="utf-8"?>
<WMS_Capabilities version="1.3.0" xmlns="http://www.opengis.net/wms" xmlns:inspire_common="https://inspire.ec.europa.eu/schemas/common/1.0" xmlns:inspire_vs="https://inspire.ec.europa.eu/schemas/inspire_vs/1.0" xmlns:sld="http://www.opengis.net/sld" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="https:/inspire.ec.europa.eu/schemas/inspire_vs/1.0 https://inspire.ec.europa.eu/schemas/inspire_vs/1.0/inspire_vs.xsd">
 <Service>
  <Name>
   WMS
  </Name>
  <Title>
   Sentinel Hub WMS service - dfki-haix-chma
  </Title>
  <Abstract>
   The Copernicus project's Sentinel satellites are revolutionizing earth observation (EO). Its free, full and open access to data with very short revisit times, high spatial resolution, and good spectral resolution are crucial for many applications. The portfolio of possible products is vast - use-cases of such a service range from plant health monitoring

### Creating a WMS request to get all dates with images from a given time span


In [30]:
token, oauth = get_api_access_token(CLIENT_ID, CLIENT_SECRET) if not token_check(token) else token, oauth
dates = oauth.request(
    url=f"https://sh.dataspace.copernicus.eu/ogc/wms/{INSTANCE_ID}?SERVICE=wms&REQUEST=GetFeatureInfo", method="GET")
dates.content

TypeError: tuple indices must be integers or slices, not str